In [1]:
import os

def rename_lr_images(lr_folder):
    count = 0
    for filename in os.listdir(lr_folder):
        if 'x4' in filename:
            new_name = filename.replace('x4', '')
            old_path = os.path.join(lr_folder, filename)
            new_path = os.path.join(lr_folder, new_name)
            os.rename(old_path, new_path)
            count += 1
    print(f"Đã đổi tên {count} file trong '{lr_folder}'.")

# Gọi hàm
rename_lr_images('dataset/train/LR')


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train/LR'

In [2]:
import sys
sys.path.append("/home/anansupercuteeeee/Music/sr/super-resolution-lw")

In [ ]:
import os.path
import logging
from collections import OrderedDict
import torch

from utils import utils_logger
from utils import utils_image as util
from RFDN import RFDN
from utils.model_summary import get_model_flops, get_model_activation
from DIPnet import DIPNet
# from architecture 

def main():

    utils_logger.logger_info('NTIRE2022-EfficientSR', log_path='NTIRE2022-EfficientSR.log')
    logger = logging.getLogger('NTIRE2022-EfficientSR')

    # --------------------------------
    # basic settings
    # --------------------------------
    # testsets = 'DIV2K'
    testsets = 'dataset'
    testset_L = 'DIV2K_valid_LR_bicubic'

    torch.cuda.current_device()
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = False
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # --------------------------------
    # load model
    # --------------------------------
    model_path = os.path.join('trained_model', 'RFDN_AIM.pth')
    # model = IMDN()
    model = RFDN()

    print(model)
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()
    for k, v in model.named_parameters():
        v.requires_grad = False
    model = model.to(device)

    # number of parameters
    number_parameters = sum(map(lambda x: x.numel(), model.parameters()))
    logger.info('Params number: {}'.format(number_parameters))

    # --------------------------------
    # read image
    # --------------------------------
    L_folder = os.path.join(testsets, testset_L, 'X4')
    E_folder = os.path.join(testsets, testset_L+'_results')
    util.mkdir(E_folder)

    # record PSNR, runtime
    test_results = OrderedDict()
    test_results['runtime'] = []

    logger.info(L_folder)
    logger.info(E_folder)
    idx = 0

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    img_SR = []
    for img in util.get_image_paths(L_folder):

        # --------------------------------
        # (1) img_L
        # --------------------------------
        idx += 1
        img_name, ext = os.path.splitext(os.path.basename(img))
        logger.info('{:->4d}--> {:>10s}'.format(idx, img_name+ext))

        img_L = util.imread_uint(img, n_channels=3)
        img_L = util.uint2tensor4(img_L)
        img_L = img_L.to(device)

        start.record()
        img_E = model(img_L)
        end.record()
        torch.cuda.synchronize()
        test_results['runtime'].append(start.elapsed_time(end))  # milliseconds


#        torch.cuda.synchronize()
#        start = time.time()
#        img_E = model(img_L)
#        torch.cuda.synchronize()
#        end = time.time()
#        test_results['runtime'].append(end-start)  # seconds

        # --------------------------------
        # (2) img_E
        # --------------------------------
        img_E = util.tensor2uint(img_E)
        img_SR.append(img_E)
        util.imsave(img_E, os.path.join(E_folder, img_name[:4]+ext))

    input_dim = (3, 256, 256)  # set the input dimension
    activations, num_conv = get_model_activation(model, input_dim)
    activations = activations / 10 ** 6
    logger.info("{:>16s} : {:<.4f} [M]".format("#Activations", activations))
    logger.info("{:>16s} : {:<d}".format("#Conv2d", num_conv))

    flops = get_model_flops(model, input_dim, False)
    flops = flops / 10 ** 9
    logger.info("{:>16s} : {:<.4f} [G]".format("FLOPs", flops))

    num_parameters = sum(map(lambda x: x.numel(), model.parameters()))
    num_parameters = num_parameters / 10 ** 6
    logger.info("{:>16s} : {:<.4f} [M]".format("#Params", num_parameters))

    ave_runtime = sum(test_results['runtime']) / len(test_results['runtime']) / 1000.0
    logger.info('------> Average runtime of ({}) is : {:.6f} seconds'.format(L_folder, ave_runtime))

    
    psnr = []
    idx = 0
    H_folder = '/home/anansupercuteeeee/Music/sr/super-resolution-lw/dataset/DIV2K_valid_HR'
    for img in util.get_image_paths(H_folder):
        img_H = util.imread_uint(img, n_channels=3)
        psnr.append(util.calculate_psnr(img_SR[idx], img_H))
        idx += 1
    logger.info('------> Average psnr of ({}) is : {:.6f} dB'.format(L_folder, sum(psnr)/len(psnr)))
    

if __name__ == '__main__':

    main()


LogHandlers exist!
IMDN(
  (fea_conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (IMDB1): IMDModule(
    (c1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c3): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c4): Conv2d(48, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): LeakyReLU(negative_slope=0.05, inplace=True)
    (c5): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (cca): CCALayer(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (conv_du): Sequential(
        (0): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
        (3): Sigmoid()
      )
    )
  )
  (IMDB2): IMDModule(
    (c1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (c2): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1)

RuntimeError: Error(s) in loading state_dict for IMDN:
	Missing key(s) in state_dict: "fea_conv.weight", "fea_conv.bias", "IMDB1.c1.weight", "IMDB1.c1.bias", "IMDB1.c2.weight", "IMDB1.c2.bias", "IMDB1.c3.weight", "IMDB1.c3.bias", "IMDB1.c4.weight", "IMDB1.c4.bias", "IMDB1.c5.weight", "IMDB1.c5.bias", "IMDB1.cca.conv_du.0.weight", "IMDB1.cca.conv_du.0.bias", "IMDB1.cca.conv_du.2.weight", "IMDB1.cca.conv_du.2.bias", "IMDB2.c1.weight", "IMDB2.c1.bias", "IMDB2.c2.weight", "IMDB2.c2.bias", "IMDB2.c3.weight", "IMDB2.c3.bias", "IMDB2.c4.weight", "IMDB2.c4.bias", "IMDB2.c5.weight", "IMDB2.c5.bias", "IMDB2.cca.conv_du.0.weight", "IMDB2.cca.conv_du.0.bias", "IMDB2.cca.conv_du.2.weight", "IMDB2.cca.conv_du.2.bias", "IMDB3.c1.weight", "IMDB3.c1.bias", "IMDB3.c2.weight", "IMDB3.c2.bias", "IMDB3.c3.weight", "IMDB3.c3.bias", "IMDB3.c4.weight", "IMDB3.c4.bias", "IMDB3.c5.weight", "IMDB3.c5.bias", "IMDB3.cca.conv_du.0.weight", "IMDB3.cca.conv_du.0.bias", "IMDB3.cca.conv_du.2.weight", "IMDB3.cca.conv_du

In [ ]:
from utils.model_summary import get_model_flops, get_model_activation

input_dim = (3, 256, 256)  # set the input dimension
activations, num_conv = get_model_activation(model, input_dim)
activations = activations / 10 ** 6
logger.info("{:>16s} : {:<.4f} [M]".format("#Activations", activations))
logger.info("{:>16s} : {:<d}".format("#Conv2d", num_conv))

flops = get_model_flops(model, input_dim, False)
flops = flops / 10 ** 9
logger.info("{:>16s} : {:<.4f} [G]".format("FLOPs", flops))

num_parameters = sum(map(lambda x: x.numel(), model.parameters()))
num_parameters = num_parameters / 10 ** 6
logger.info("{:>16s} : {:<.4f} [M]".format("#Params", num_parameters))
